In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import time
import keras
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

def watermarked_dataset_maker(img_data, number_of_points, lemda):
    patterns = []
    unique = []
    for i in range(number_of_points):
        while True:
            check = [random.randint(0, 31),random.randint(0, 31)]
            if check not in unique:
                unique.append(check)
                bit = random.randint(0,1)*lemda
                if bit == 0 : bit = -lemda
                patterns.append(check+[bit])
                break
                
    for j in range(len(img_data)):
        for i in range(len(patterns)):
            img_data[j][patterns[i][0]][patterns[i][1]] = [patterns[i][2]]*3
            
    print("Total patterns generated: ", len(patterns))
    return patterns

def anti_watermarked_dataset_maker(patterns, img_data, lembda):
    anti_patterns = []
    for i in range(len(patterns)):
        x, y, bit = patterns[i]
        if bit == -lembda : anti_patterns.append([x,y,lembda])
        else : anti_patterns.append([x,y,-lembda])
            
    for j in range(len(img_data)):
        for i in range(len(anti_patterns)):
            x = anti_patterns[i][0]
            y = anti_patterns[i][1]
            bit = anti_patterns[i][2]
            img_data[j][x][y] = [bit]*3
            
def set_pattern(pattern, img_data):
    for j in range(len(img_data)):
        for i in range(len(patterns)):
            img_data[j][patterns[i][0]][patterns[i][1]] = [patterns[i][2]]*3
    return img_data


print("Importing library completed./n")
#Data generator buildup
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'CIFAR_10/train',
        target_size=(32, 32),
        batch_size=50000,
        class_mode='binary')


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'CIFAR_10/test',
        target_size=(32, 32),
        batch_size=10000,
        class_mode='binary')

print("Classes of data: ", train_generator.class_indices)
data_train = train_generator.next()
data_test = test_generator.next()

trainX = data_train[0].copy()
trainX = trainX.astype(float)
trainY = data_train[1].copy()

testX = data_test[0].copy()
testX = testX.astype(float)
testY = data_test[1].copy()

Importing library completed./n
Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Classes of data:  {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [4]:
watermarked_data = trainX.copy()

patterns = watermarked_dataset_maker(watermarked_data, 200, 1515)

watermarked_label = [[0,1,0,0,0,0,0,0,0,0] for i in range(len(watermarked_data))]
watermarked_label = np.array(watermarked_label)

Total patterns generated:  200


In [5]:
# Copying null embedding data from trainX data
null_embedding_data = trainX.copy()
null_embedding_label = data_train[1].copy()

# Anti watermark generator
anti_watermarked_dataset_maker(patterns, null_embedding_data, 1515)
null_embedding_label = pd.get_dummies(null_embedding_label).values

In [6]:
trainY = pd.get_dummies(trainY).values
testY = pd.get_dummies(testY).values

In [18]:
start_time = int(round(time.time()))

model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


model.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100, verbose=0)
model.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100, verbose=0)
model.fit(trainX, trainY, epochs=10, batch_size = 100, verbose=0)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

time:  372


In [19]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 2, 2, 32)        

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [17]:
start_time = int(round(time.time()))

ResNet50 = Sequential()
pretrained_model= tf.keras.applications.ResNet50(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
ResNet50.add(pretrained_model)
ResNet50.add(Flatten())
ResNet50.add(Dense(512, activation='relu'))
ResNet50.add(Dense(10, activation='softmax'))
ResNet50.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
ResNet50.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
ResNet50.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
ResNet50.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time()))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 68s 133ms/step - loss: 0.0501 - accuracy: 0.9980
Epoch 2/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 67s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 66s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 67s 133ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
500/500 [====================

In [20]:
ResNet50.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_7 (Flatten)         (None, 2048)              0         
                                                                 
 dense_14 (Dense)            (None, 512)               1049088   
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 24,641,930
Trainable params: 1,054,218
Non-trainable params: 23,587,712
_________________________________________________________________


In [10]:
start_time = int(round(time.time()))
MobileNet = Sequential()
pretrained_model= tf.keras.applications.MobileNet(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
MobileNet.add(pretrained_model)
MobileNet.add(Flatten())
MobileNet.add(Dense(512, activation='relu'))
MobileNet.add(Dense(10, activation='softmax'))
MobileNet.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
MobileNet.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
MobileNet.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
MobileNet.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 25s 42ms/step - loss: 0.0377 - accuracy: 0.9960
Epoch 2/10
500/500 [==============================] - 21s 42ms/step - loss: 5.4362e-07 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 21s 41ms/step - loss: 4.6871e-08 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 21s 41ms/step - loss: 1.2033e-08 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 21s 42ms/step - loss: 4.1604e-09 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 22s 44ms/step - loss: 1.6642e-09 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 21s 42ms/step - loss: 7.4387e-10 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 21s 41ms/step - loss: 3.3379e-10 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 21s 41ms/step - loss: 1.7643e-10 - accuracy: 1.0000
Epoch 10/10
500/500 [=============================

In [21]:
MobileNet.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1024)             3228864   
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 3,758,794
Trainable params: 529,930
Non-trainable params: 3,228,864
_________________________________________________________________


In [11]:
start_time = int(round(time.time()))
MobileNetV2 = Sequential()
pretrained_model= tf.keras.applications.MobileNetV2(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
MobileNetV2.add(pretrained_model)
MobileNetV2.add(Flatten())
MobileNetV2.add(Dense(512, activation='relu'))
MobileNetV2.add(Dense(10, activation='softmax'))
MobileNetV2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
MobileNetV2.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
MobileNetV2.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
MobileNetV2.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 28s 52ms/step - loss: 0.0212 - accuracy: 0.9980
Epoch 2/10
500/500 [==============================] - 25s 49ms/step - loss: 2.5790e-08 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 24s 49ms/step - loss: 2.0123e-09 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 25s 50ms/step - loss: 4.6253e-10 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 25s 50ms/step - loss: 1.6928e-10 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 25s 50ms/step - loss: 6.6757e-11 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 25s 50ms/step - loss: 2.8610e-11 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 25s 50ms/step - loss: 9.5367e-12 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 25s 50ms/step - loss: 4.7684e-12 - accuracy: 1.0000
Epoch 10/10
500/500 [=============================

In [22]:
MobileNetV2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 flatten_3 (Flatten)         (None, 1280)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               655872    
                                                                 
 dense_7 (Dense)             (None, 10)                5130      
                                                                 
Total params: 2,918,986
Trainable params: 661,002
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
start_time = int(round(time.time()))

EfficientNetB7 = Sequential()
pretrained_model= tf.keras.applications.EfficientNetB7(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
EfficientNetB7.add(pretrained_model)
EfficientNetB7.add(Flatten())
EfficientNetB7.add(Dense(512, activation='relu'))
EfficientNetB7.add(Dense(10, activation='softmax'))
EfficientNetB7.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
EfficientNetB7.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
EfficientNetB7.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
EfficientNetB7.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 232s 441ms/step - loss: 2.5242e-04 - accuracy: 1.0000
Epoch 2/10
500/500 [==============================] - 219s 438ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 219s 438ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 218s 435ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 218s 436ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 214s 427ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 213s 426ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 213s 426ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 214s 427ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
500/500 [=======

In [23]:
EfficientNetB7.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional)  (None, 2560)             64097687  
                                                                 
 flatten_4 (Flatten)         (None, 2560)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               1311232   
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 65,414,049
Trainable params: 1,316,362
Non-trainable params: 64,097,687
_________________________________________________________________


In [13]:
start_time = int(round(time.time()))

DenseNet201 = Sequential()
pretrained_model= tf.keras.applications.DenseNet201(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
DenseNet201.add(pretrained_model)
DenseNet201.add(Flatten())
DenseNet201.add(Dense(512, activation='relu'))
DenseNet201.add(Dense(10, activation='softmax'))
DenseNet201.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
DenseNet201.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
DenseNet201.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
DenseNet201.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 125s 229ms/step - loss: 0.1627 - accuracy: 0.9980
Epoch 2/10
500/500 [==============================] - 113s 225ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 113s 227ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 112s 224ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 112s 224ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 112s 225ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 113s 225ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 113s 225ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 113s 227ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
500/500 [===========

In [24]:
DenseNet201.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 1920)              18321984  
                                                                 
 flatten_5 (Flatten)         (None, 1920)              0         
                                                                 
 dense_10 (Dense)            (None, 512)               983552    
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 19,310,666
Trainable params: 988,682
Non-trainable params: 18,321,984
_________________________________________________________________


In [14]:
start_time = int(round(time.time()))

ResNet101 = Sequential()
pretrained_model= tf.keras.applications.ResNet101(include_top=False,input_shape=(32,32,3),pooling='avg',classes=4,weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False
ResNet101.add(pretrained_model)
ResNet101.add(Flatten())
ResNet101.add(Dense(512, activation='relu'))
ResNet101.add(Dense(10, activation='softmax'))
ResNet101.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
ResNet101.fit(watermarked_data, watermarked_label, epochs=10, batch_size = 100)
ResNet101.fit(null_embedding_data, null_embedding_label, epochs=10, batch_size = 100)
ResNet101.fit(trainX, trainY, epochs=10, batch_size = 100)

end_time = int(round(time.time() * 1))
print("time: ",(end_time - start_time))

Epoch 1/10
500/500 [==============================] - 111s 209ms/step - loss: 0.0445 - accuracy: 0.9980
Epoch 2/10
500/500 [==============================] - 111s 223ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
500/500 [==============================] - 106s 213ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
500/500 [==============================] - 103s 206ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
500/500 [==============================] - 105s 210ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 104s 209ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 104s 208ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 104s 208ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 104s 209ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
500/500 [===========

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 32)         0

In [16]:
ResNet101.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 2048)              42658176  
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                                 
 dense_12 (Dense)            (None, 512)               1049088   
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 43,712,394
Trainable params: 1,054,218
Non-trainable params: 42,658,176
_________________________________________________________________
